# Assigment 3

I write code to scrap poems for biggerst turkish poem website which is antoloji.com

In [88]:
import scrapy

class Antoloji(scrapy.Spider):
    name = 'Antoloji'
    url = 'https://www.antoloji.com'
    poet = 'oktay-rifat'

    def start_requests(self):
        yield scrapy.Request('%s/%s/siirleri/' % (self.url, self.poet),
                             self.parse_all)

    def parse_all(self, response):
        for i in range(1, len(response.css('.pagination li')) - 1):
            yield scrapy.Request('%s/%s/siirleri/ara-/sirala-/sayfa-%s/' %
                                 (self.url, self.poet, i), self.parse_list)

    def parse_list(self, response):
        for i in response.css('.poemListBox a::attr(href)').extract():
            yield scrapy.Request(self.url + i, self.parse)

    def parse(self, response):
        with open('poets/%s.txt' % self.poet, 'a') as f:
            text = ''.join(response.css('.pd-text p::text').extract())
            f.write(text)
            yield {'poem': text}

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, Dropout, SimpleRNN, GRU
from keras.callbacks import ModelCheckpoint

import numpy as np
import random
import pickle

I implement textgenerator with keras

In [90]:
class TextGenerator:
    def __init__(self, seq_len=50, temperature=0.8, epoch=50, rnn_layer=LSTM):
        self.seq_len = seq_len
        self.temperature = temperature
        self.epochs = epoch
        self.rnn_layer = rnn_layer

    def read_data(self, files_paths):
        self.text = '\n'.join(open(path).read()
                              for path in files_paths).lower()
        self.set_vocab_mapping(self.text)
        return self.text_vec(self.text)

    def text_vec(self, text):
        sentences = []
        next_chars = []
        for i in range(0, len(text) - self.seq_len):
            sentences.append(text[i:i + self.seq_len])
            next_chars.append(text[i + self.seq_len])
        
        X = np.zeros(
            (len(sentences), self.seq_len, self.vocab_len), dtype=np.bool)
        y = np.zeros((len(sentences), self.vocab_len), dtype=np.bool)
        for i, sentence in enumerate(sentences):
            for t, char in enumerate(sentence):
                X[i, t, self.char_indices[char]] = 1
            y[i, self.char_indices[next_chars[i]]] = 1
        return X, y

    def set_vocab_mapping(self, text):
        chars = sorted(list(set(text)))
        self.vocab_len = len(chars)
        self.char_indices = {c: i for i, c in enumerate(chars)}
        self.indices_char = {i: c for i, c in enumerate(chars)}

    def build_nn(self):
        print(self.seq_len, self.vocab_len)
        model = Sequential()
        model.add(self.rnn_layer(256, input_shape=(self.seq_len, self.vocab_len)))
        model.add(Dropout(0.2))
        model.add(Dense(self.vocab_len))
        model.add(Activation('softmax'))
        model.compile(loss='categorical_crossentropy', optimizer='adam')
        self.model = model

    def fit(self, X, y):
        self.build_nn()
        filepath = "models/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
        checkpoint = ModelCheckpoint(
            filepath,
            monitor='loss',
            verbose=1,
            save_best_only=True,
            mode='min')
        self.model.fit(
            X, y, epochs=self.epochs, batch_size=128, callbacks=[checkpoint])

    def diversity(self, preds):
        preds = np.asarray(preds).astype('float64')
        preds = np.log(preds) / self.temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        probas = np.random.multinomial(1, preds, 1)
        return np.argmax(probas)

    def predict(self, X):
        return self.model.predict(X, verbose=0)[0]

    def generate(self):                                                                                                                                                                                             
        sentence = self.get_random_sentence()                                                                                                                                                                       
        generated = list()                                                                                                                                                                                          
        for i in range(400):                                                                                                                                                                                        
            X, _ = self.text_vec(sentence)                                                                                                                                                                          
            next_index = self.diversity(self.predict(X))                                                                                                                                                            
            next_char = self.indices_char[next_index]                                                                                                                                                               
            sentence = sentence[1:] + next_char                                                                                                                                                                     
            generated.append(next_char)                                                                                                                                                                             
        return ''.join(generated)                                                                                                                                                                                   
     
    def get_random_sentence(self):
        start_index = random.randint(0, len(self.text) - self.seq_len - 1)
        return self.text[start_index:start_index + self.seq_len + 1]

    def load(self, path):
        self.model.load_weights(path)


SimpleRnn results do not make sense.

In [5]:
from IPython.display import clear_output

text_generator = TextGenerator(rnn_layer=SimpleRNN, epoch=20)
X, y = text_generator.read_data(['poets/sezai-karakoc.txt'])
text_generator.fit(X, y)

clear_output()
print(text_generator.generate())

ğuu  koipşleerdii

b ut uk uçdıuğ ubçaağ ıölmüeşreezlierr ebtii  öoltuosyuas aallıağnıuz ashkell igbiil e dbeenciel ekkeis ebseinm eşrtim
aaüy ok
urmaas ıullkaıt ıglea  k ibkiinç ek lgaüztcee
 p,e lgeel een eks isçteun ekntiarcaı 
yseenlier  iȧ kvaurllaartıu,  kbeer ebnia  vç  kbael abdeık  beanz ı
çveep eyneen lbeirnt iiemt etn ikçdiu
myean  k agğiı  gaien  as tiabniaylaa rıollaarn  ös üünrüey e


GRU is not looks well too

In [7]:
from IPython.display import clear_output

text_generator = TextGenerator(rnn_layer=GRU, epoch=20)
X, y = text_generator.read_data(['poets/sezai-karakoc.txt'])
text_generator.fit(X, y)

clear_output()
print(text_generator.generate())

şyüü kköarleırl
ıgşıaml aknlaam ıbşaadtımr aelsein
bbiurtmuaşctamlaarlıağlıa ykaen ekdtii
çbeirtiirlie  adkeıllaıd akc agkıınmdaannıeş ıprdaanl agtıamn eslkii  açkıır
agyaalt ukğuunmuaşlkae skaartlıa mdiemkeer einkdeilmaezlierleedleer
edneim  öblüey oblua msaantaırlaarckıımr
aasyıalnaızm aşckıssıaz

aacmaanl adcaakl akr agliayr ıgkaaylıa ykaanm adyaaltaın çkoılrma  m
esrkiiş
teenm eşştiil mdeun


In [6]:
from IPython.display import clear_output

text_generator = TextGenerator(rnn_layer=LSTM, epoch=20)
X, y = text_generator.read_data(['poets/sezai-karakoc.txt'])
text_generator.fit(X, y)

clear_output()
print(text_generator.generate())

ğuu  koipşleerdii

b ut uk uçdıuğ ubçaağ ıölmüeşreezlierr ebtii  öoltuosyuas aallıağnıuz ashkell igbiil e dbeenciel ekkeis ebseinm eşrtim
aaüy ok
urmaas ıullkaıt ıglea  k ibkiinç ek lgaüztcee
 p,e lgeel een eks isçteun ekntiarcaı 
yseenlier  iȧ kvaurllaartıu,  kbeer ebnia  vç  kbael abdeık  beanz ı
çveep eyneen lbeirnt iiemt etn ikçdiu
myean  k agğiı  gaien  as tiabniaylaa rıollaarn  ös üünrüey e


Result are too bad. Probably there is bug in my implementation because I obtain better result with using other implentation from internet but I could not find the bug alhough I debug it for a day.